## tidied up version of convex mesh generation code

In [28]:
from skimage import io, data
from skimage import filters, measure
import napari
from scipy import ndimage as ndi
import numpy as np
from PIL import Image
from aicsimageio import AICSImage, imread
import cv2

In [29]:
from scipy.spatial import ConvexHull, Delaunay

In [30]:
import matplotlib.path as p

In [80]:
viewer = napari.Viewer(title='convex_hull_surface')

In [32]:
image_1_name=str('i_ch1')
layer_1_name=image_1_name
image_2_name=str('i_ch2')
layer_2_name=image_2_name
file_path=str('C:/Users/hkqur/Documents/napari/eb3/2024-02-09/fast/eb3_lifeact_58hrs_at_26C_e3_fast_proj-01-Airyscan Processing-17.czi')

In [33]:
image_i = AICSImage(file_path)
print(image_i.shape)

(114, 1, 26, 332, 332)


In [34]:
i_ch1 = image_i.data[:,0,:,:]
#i_ch2 = image_i.data[:,1,:,:]


In [35]:
viewer.add_image(i_ch1, name='i_ch1',colormap='magenta',blending='additive')
viewer.layers['i_ch1'].scale=[6,1,1]
'''
viewer.add_image(i_ch2, name='i_ch2',colormap='green',blending='additive')
viewer.layers['i_ch2'].scale=[6,1,1]
'''

"\nviewer.add_image(i_ch2, name='i_ch2',colormap='green',blending='additive')\nviewer.layers['i_ch2'].scale=[6,1,1]\n"

### convex hull generation

In [36]:
convex_points=viewer.layers['Points'].data

In [37]:
relevant_indices=[1,2,3]
convex_points_3D = [convex_points[i] for i in relevant_indices]

In [38]:
array_3D = []
for a in range(0,len(convex_points)):
    array_3D_element = [convex_points[a][i] for i in relevant_indices]
    print(array_3D_element)
    array_3D.append(array_3D_element)

[0.0, 154.0173485195182, 193.47593645280685]
[0.0, 135.22754777572578, 178.86164698541276]
[0.0, 103.49366207509861, 191.3881808146077]
[0.0, 77.18794103378923, 222.28696425995523]
[0.0, 77.60549216142905, 239.40656049318832]
[0.0, 100.57080418161979, 266.96493491741717]
[0.0, 118.52550267013252, 299.53392287332406]
[0.0, 153.18224626423853, 284.91963340592997]
[0.0, 173.22470039095043, 270.3053439385359]
[0.0, 176.14755828442927, 237.73635598262902]
[0.0, 170.30184249747163, 213.10083945187895]
[7.0, 146.50142822200124, 198.068998856845]
[7.0, 111.42713350025541, 186.37756728292973]
[7.0, 43.366299694962876, 218.52900411119668]
[7.0, 76.35283877850956, 193.89348758044667]
[7.0, 34.18017488688659, 260.7016680028197]
[7.0, 47.5418109713612, 284.50208227829006]
[7.0, 65.91406058751377, 301.62167851152316]
[7.0, 88.04427035242485, 318.3237236171164]
[7.0, 109.75692898969608, 321.66413263823506]
[7.0, 128.54672973348852, 323.33433714879436]
[7.0, 161.53326881703518, 296.19351385220534]
[7.

In [39]:
cell_hull=ConvexHull(array_3D)

In [40]:
cell_convex_test=cell_hull.simplices #generate vertex order of convex hull

In [41]:
surface=viewer.add_surface(data=(convex_points,cell_convex_test)) #cell_convex_test is the points data, convex points is how to join them to create polygon
surface.normals.face.visible = True
surface.normals.vertex.visible = True
surface.wireframe.visible = True

In [42]:
viewer.layers['Surface'].scale=[6,1,1]

In [43]:
print(cell_hull.volume)

424473.4555700367


C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms


### create 3D mask from convex hull

In [44]:
conv_points = cell_hull.points

In [45]:
deln = Delaunay(conv_points[cell_hull.vertices])

In [46]:
#print(i_ch1.shape)
i_shape = i_ch1[0,:,:,:]
print(i_shape.shape)

i_shape2 = np.moveaxis(i_shape,0,2)
print(i_shape2.shape)

idx_2d = np.indices(i_shape.shape[1:])
print(idx_2d.shape)
idx_2d = np.moveaxis(idx_2d,0,-1)
print(idx_2d.shape)

(26, 332, 332)
(332, 332, 26)
(2, 332, 332)
(332, 332, 2)


In [47]:
print(i_shape.ndim)
print(*i_shape.shape)


3
26 332 332


In [48]:
idx_3d = np.zeros((*i_shape.shape[1:3],i_shape.ndim))
idx_3d.shape
idx_3d[:,:,1:] = idx_2d

In [49]:
mask=np.zeros_like(i_shape,dtype=bool)

In [50]:
mask.shape

(26, 332, 332)

In [51]:
idx_3d.shape

(332, 332, 3)

In [52]:
len(i_shape)

26

In [53]:
for z in range(len(i_shape)):
    idx_3d[:,:,0] = z
    s = deln.find_simplex(idx_3d)
    mask[z,(s != -1)] = 1

In [82]:
viewer.add_image(mask, name='mask',colormap='magenta',blending='additive')
viewer.layers['mask'].scale=[6,1,1]

### crop on mask

In [59]:
mask.shape

(26, 332, 332)

In [60]:
i_shape.shape

(26, 332, 332)

In [61]:
mask_int = mask*1


In [62]:
masked_crop = np.zeros(i_shape2.shape)

In [63]:
masked_crop.shape

(332, 332, 26)

In [64]:
i_shape2.dtype

dtype('uint16')

In [65]:
masked_crop = masked_crop.astype(int)

In [66]:
mask_int.dtype

dtype('int32')

In [67]:
mask_int_uint = np.uint16(mask_int)
mask_crop_uint = np.uint16(masked_crop)


In [68]:
mask.shape
i_ch1.shape

(114, 26, 332, 332)

In [84]:
i_ch1_tp0=i_ch1[0,:,:,:]
#i_ch1_tp0 = np.moveaxis(i_ch1_tp0,0,-1)
i_ch1_tp0.shape

(26, 332, 332)

In [77]:
print('{} = mask_crop_uint  {} = i_ch1_tp0'.format(mask_crop_uint.shape,(np.flip(i_ch1_tp0)).shape))

(332, 332, 26) = mask_crop_uint  (332, 332, 26) = i_ch1_tp0


In [85]:
mask_i_ch1_tp0 = mask_int_uint*i_ch1_tp0

In [89]:
viewer.add_image(mask_i_ch1_tp0,name='mask_crop')
viewer.layers['mask_crop'].scale = [6,1,1]

In [140]:

cv2.bitwise_and(mask_int_uint,i_shape,mask_crop_uint)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [151]:
mask_crop_uint = cv2.bitwise_and(mask_int_uint,i_ch1)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'


C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms
C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms
C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms
C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms
C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: invalid value encountered in divide
  unit_normals = normals / norms
C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\vispy\visuals\mesh_normals.py:129: RuntimeWarning: i

In [148]:
mask_crop_uint.shape

(26, 332, 332)

In [149]:
viewer.add_image(mask_crop_uint,name='mask_crop')
viewer.layers['mask_crop'].scale=[6,1,1]